In [2]:

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.context import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import lit, udf
import pyspark.sql.types as T 
import argparse
from configparser import ConfigParser
import json

conf = SparkConf()
conf.setAppName('pillar')
# sc = SparkContext(conf=conf)
sc = SparkContext.getOrCreate(conf = conf);
spark = SparkSession(sc)
sc.setLogLevel('WARN')
sql_context = SQLContext(sc)
spark = SparkSession.builder.appName('pillar').getOrCreate()

In [8]:
df = spark.read.parquet("input.parquet/*")

In [4]:
# op = spark.read.parquet("output.parquet/*")
# op.toPandas()

,item_name,age,gender,weighted_sum
0,Pop Goes the Weasel2,2,female,1.66


In [9]:
df.toPandas()

,device_id,item_name,PercentPlayed,weight
0,1,Pop Goes the Weasel2,2.0,1.00
1,1,Pop Goes the Weasel2,0.5,0.66


In [10]:
genderage_df = spark.read.csv('profile.csv', header=True)
df = df.join(genderage_df, 'device_id', 'inner')

df = df.select('item_name', 'weight', 'age', 'gender')
df = df.groupBy(['item_name', 'age', 'gender']).agg(df.sum("weight").alias("weighted_sum"))
df.show()

AttributeError: 'DataFrame' object has no attribute 'sum'

In [5]:
def main(inputDir,profileMap,outputDir,config):
    profile = spark.read.csv(profileMap, header='true')
    df = spark.read.parquet(inputDir+"/*")
    df = df.join(profile, 'device_id')
    df = df.select('item_name', 'weight', 'age', 'gender')
    df = df.groupBy(['item_name', 'age', 'gender']).sum().alias('weighted_sum')
    df.write.parquet(outputDir)

In [6]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser() 
    parser.add_argument('-i', '--input', required=True)
    parser.add_argument('-m', '--map', required=True)
    parser.add_argument('-o', '--output', required=True)
    parser.add_argument('-c', '--config', required=True)
    args = parser.parse_args()
    config = ConfigParser()
    config.read(args.config)
    main(args.input,args.map,args.output,config)

usage: ipykernel_launcher.py [-h] -i INPUT -m MAP -o OUTPUT -c CONFIG
ipykernel_launcher.py: error: the following arguments are required: -i/--input, -m/--map, -o/--output, -c/--config


SystemExit: 2

/Users/elaine/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
